# HALI Data Analysis

#### Estimated intervention effects under five different GEE approaches (with robust SE and exchangeable working correlation matrix)

* Draw table 1B

* Draw table S1

In [54]:
table_1b

Method,Estimated,SE,Confidence Interval,p-value
CRA-GEE,1.82,0.19,"(1.26, 2.65)",0.002
A-CRA-GEE,1.93,0.17,"(1.38, 2.71)",<0.001
W-GEE,1.80,0.19,"(1.23, 2.62)",0.002
CW-GEE,1.83,0.19,"(1.26, 2.65)",0.001
MMI-GEE,1.81,0.19,"(1.25, 2.63)",0.002


In [51]:
table_s1

Method,Estimated,SE,Confidence Interval,p-value,Estimated,SE,Confidence Interval,p-value
CRA-GEE,1.82,0.19,"(1.26, 2.65)",0.002,1.84,0.19,"(1.27, 2.66)",0.001
A-CRA-GEE,1.93,0.17,"(1.38, 2.71)",<0.001,1.92,0.17,"(1.37, 2.68)",<0.001
W-GEE,1.80,0.19,"(1.23, 2.62)",0.002,1.81,0.19,"(1.24, 2.63)",0.002
CW-GEE,1.83,0.19,"(1.26, 2.65)",0.001,1.83,0.19,"(1.26, 2.65)",0.002
MMI-GEE,1.81,0.19,"(1.25, 2.63)",0.002,1.79,0.19,"(1.23, 2.60)",0.002


# Code Detail

In [1]:
# install.packages('readstata13')
# install.packages('geeM')
# install.packages('jomo')
# install.packages('knitr')

In [3]:
library(readstata13) # library for read in the dataset
library(lme4) # library for GLM
library(geeM)
library(jomo)
library(knitr)

### Read in data

the full dataset named dat

In [4]:
setwd('/Users/yaolanqiu/Documents/HALI/DATA')
dat <- read.dta13('HALI_CLASS1_2539_MASTER_EDUC_FU1_FU2_LONG_accounts_withdrawals_BL_AS_COV_AND_OTHER_COV_17.11.2016.dta',
                  nonint.factors = TRUE, generate.factors=TRUE)

In [5]:
dim(dat)

[1] 5078  220

### we only focused on 9 month data

the data with only 9 month named data0

In [6]:
data0 <- dat[dat$visit=='9-month FU',]
dim(data0) # 2539 220

[1] 2539  220

### select the input variables

In [7]:
dat0 <- data0[,c("school_id","LIT_grp",
                 "BL_gll21_total", 'age_child','sex',
                 'schlevel_comp',"BL_ses",
                 "gll21_total")]
rownames(dat0) <- NULL
dim(dat0) # 2539 8

[1] 2539    8

### remove the data with missing covariates 

In [8]:
dat1 = dat0[(is.na(dat0$school_id) == 0 & 
      is.na(dat0$LIT_grp) == 0 &
      is.na(dat0$BL_gll21_total) == 0 &
      is.na(dat0$age_child)== 0 & 
      is.na(dat0$sex) == 0 & 
      is.na(dat0$schlevel_comp) == 0 &
      is.na(dat0$BL_ses) ==0) ,]
rownames(dat1) = NULL
dim(dat1) # 2465 8

[1] 2465    8

# Draw table 1b

#### The formula we will use:



In [9]:
# formula for GEE
formula1 = y ~ LIT_grp
formula2 = y ~ LIT_grp + BL_gll21_total + 
               age_child + sex + schlevel_comp + BL_ses

# formula to calculate the weights 
formula3 = missing ~ LIT_grp + BL_gll21_total + 
                     age_child + sex + schlevel_comp + 
                     BL_ses
formula4 = missing ~ LIT_grp + BL_gll21_total + 
                     age_child + sex + schlevel_comp + 
                     BL_ses + (1 | school_id)

Add misisng indicator

In [10]:
dat1$missing = is.na(dat1$gll21_total)

High literacy indicator

In [11]:
y = ifelse(dat1$gll21_total>10,1,0)
dat1$y = y

 Empty vectors that save results later

In [12]:
means <- c();sds <- c();cis <- c();ps <- c()

## CRA-GEE

#### independent working correlation matrix 

In [13]:
ucra_ind <- geem(formula1,id = school_id, 
                 data = dat1,
                 family =  binomial("logit"),
                 corstr = "independence")
summary(ucra_ind)

            Estimates Model SE Robust SE   wald        p
(Intercept)   -0.2133   0.0609    0.1184 -1.801 0.071640
LIT_grpyes     0.6079   0.0870    0.1887  3.221 0.001275

 Estimated Correlation Parameter:  0 
 Correlation Structure:  independence 
 Est. Scale Parameter:  1.001 

 Number of GEE iterations: 2 
 Number of Clusters:  101    Maximum Cluster Size:  27 
 Number of observations with nonzero weight:  2170 

In [14]:
# save the results 
m <- summary(ucra_ind)$beta[2]
s <- summary(ucra_ind)$se.robust[2]
p <- summary(ucra_ind)$p[2]
ciu <- m-1.96*s; cil <- m+1.96*s

# save results 
means <- c(means,exp(m))
sds <- c(sds,s)
ps <- c(ps,p)
ciu <- formatC(exp(ciu),2,format="f"); cil <- formatC(exp(cil),2,format="f")
cis <- c(cis,paste('(',ciu,', ',cil,')',sep=''))

#### exchangeable working correlation matrix

In [15]:
ucra_ex <- geem(formula = formula1,id = school_id, 
                data = dat1,
                family =  binomial("logit"),
                corstr = "exchangeable")
summary(ucra_ex)

            Estimates Model SE Robust SE   wald        p
(Intercept)   -0.2196   0.1316    0.1181 -1.859 0.063020
LIT_grpyes     0.6013   0.1869    0.1899  3.166 0.001545

 Estimated Correlation Parameter:  0.1759 
 Correlation Structure:  exchangeable 
 Est. Scale Parameter:  1 

 Number of GEE iterations: 3 
 Number of Clusters:  101    Maximum Cluster Size:  27 
 Number of observations with nonzero weight:  2170 

In [16]:
# save the results 
m <- summary(ucra_ex)$beta[2]
s <- summary(ucra_ex)$se.robust[2]
p <- summary(ucra_ex)$p[2]
ciu <- m-1.96*s; cil <- m+1.96*s

# save results 
means <- c(means,exp(m))
sds <- c(sds,s)
ps <- c(ps,p)
ciu <- formatC(exp(ciu),2,format="f"); cil <- formatC(exp(cil),2,format="f")
cis <- c(cis,paste('(',ciu,', ',cil,')',sep=''))

## A-CRA-GEE

#### independent working correlation matrix

In [17]:
cra_ind <- geem(formula = formula2,
                id = school_id, data = dat1,
                family =  binomial("logit"),
                corstr = "independence")
summary(cra_ind)

                           Estimates Model SE Robust SE    wald         p
(Intercept)                 -3.35600  0.32660   0.37440 -8.9620 0.0000000
LIT_grpyes                   0.65240  0.10530   0.17090  3.8180 0.0001344
BL_gll21_total               0.33690  0.01612   0.01891 17.8200 0.0000000
age_child                    0.04344  0.03160   0.03574  1.2150 0.2242000
sexFemale                   -0.04238  0.10510   0.11320 -0.3742 0.7083000
schlevel_compprimary         0.15010  0.12020   0.12410  1.2100 0.2262000
schlevel_compsecondary       0.59720  0.20610   0.21670  2.7560 0.0058470
schlevel_compcollege/deree   0.98280  0.34330   0.38300  2.5660 0.0103000
BL_sesPoor                  -0.07301  0.15890   0.16200 -0.4506 0.6523000
BL_sesMedian poor           -0.11680  0.16140   0.18990 -0.6147 0.5387000
BL_sesLess poor             -0.06672  0.16770   0.15890 -0.4198 0.6746000
BL_sesLeast poor             0.32660  0.17900   0.18570  1.7580 0.0787100

 Estimated Correlation Parameter:  0 

In [18]:
# save the results 
m <- summary(cra_ind)$beta[2]
s <- summary(cra_ind)$se.robust[2]
p <- summary(cra_ind)$p[2]
ciu <- m-1.96*s; cil <- m+1.96*s

# save results 
means <- c(means,exp(m))
sds <- c(sds,s)
ps <- c(ps,p)
ciu <- formatC(exp(ciu),2,format="f"); cil <- formatC(exp(cil),2,format="f")
cis <- c(cis,paste('(',ciu,', ',cil,')',sep=''))

#### exchangeable working correlation matrix

In [19]:
cra_ex <- geem(formula = formula2,
               id = school_id, data = dat1,
               family =  binomial("logit"),
               corstr = "exchangeable")
summary(cra_ex)

                           Estimates Model SE Robust SE    wald         p
(Intercept)                 -3.26900  0.33530   0.36370 -8.9880 0.0000000
LIT_grpyes                   0.65910  0.15260   0.17180  3.8370 0.0001244
BL_gll21_total               0.31640  0.01607   0.01799 17.5900 0.0000000
age_child                    0.05289  0.03128   0.03328  1.5900 0.1119000
sexFemale                   -0.03572  0.09951   0.10440 -0.3421 0.7323000
schlevel_compprimary         0.12190  0.11460   0.11760  1.0360 0.3001000
schlevel_compsecondary       0.54240  0.19540   0.20070  2.7020 0.0068930
schlevel_compcollege/deree   1.03300  0.32730   0.35470  2.9120 0.0035970
BL_sesPoor                  -0.05516  0.15010   0.15920 -0.3465 0.7290000
BL_sesMedian poor           -0.12950  0.15370   0.18580 -0.6967 0.4860000
BL_sesLess poor             -0.09199  0.16010   0.15440 -0.5958 0.5513000
BL_sesLeast poor             0.21320  0.17220   0.17430  1.2230 0.2212000

 Estimated Correlation Parameter:  0.

In [20]:
# save the results 
m <- summary(cra_ex)$beta[2]
s <- summary(cra_ex)$se.robust[2]
p <- summary(cra_ex)$p[2]
ciu <- m-1.96*s; cil <- m+1.96*s

# save results 
means <- c(means,exp(m))
sds <- c(sds,s)
ps <- c(ps,p)
ciu <- formatC(exp(ciu),2,format="f"); cil <- formatC(exp(cil),2,format="f")
cis <- c(cis,paste('(',ciu,', ',cil,')',sep=''))

## W-GEE

#### calculate the weights

In [21]:
w1 <- glm(formula = formula3, data = dat1,
          family = binomial(link='logit'))
w2 <- glmer(formula = formula4, data = dat1,
            family = binomial(link='logit'),
            control=glmerControl(optimizer="bobyqa",
                    optCtrl=list(maxfun=2e5)))
# increase the iteration times to avoid non-convergence

In [22]:
w1 <- predict(w1,type="response")  # get the weights value from the glm
w2 <- predict(w2,type="response")  # get the weights value form the glmer
# add the weights to the dataset
dat1$weight <- (1/ w1)
dat1$weight2 <- (1/w2)

#### independent working correlation matrix

In [23]:
ipw_ind <- geem(formula = y~LIT_grp,
                id = school_id, data = dat1,
                family =  binomial("logit"),
                weights = dat1$weight,
                corstr = "independence")
summary(ipw_ind)

            Estimates Model SE Robust SE    wald        p
(Intercept)   -0.0699  0.05971    0.1183 -0.5907 0.554700
LIT_grpyes     0.5918  0.08753    0.1914  3.0930 0.001983

 Estimated Correlation Parameter:  0 
 Correlation Structure:  independence 
 Est. Scale Parameter:  8.782 

 Number of GEE iterations: 2 
 Number of Clusters:  101    Maximum Cluster Size:  27 
 Number of observations with nonzero weight:  2170 

In [24]:
# save the results 
m <- summary(ipw_ind)$beta[2]
s <- summary(ipw_ind)$se.robust[2]
p <- summary(ipw_ind)$p[2]
ciu <- m-1.96*s; cil <- m+1.96*s

# save results 
means <- c(means,exp(m))
sds <- c(sds,s)
ps <- c(ps,p)
ciu <- formatC(exp(ciu),2,format="f"); cil <- formatC(exp(cil),2,format="f")
cis <- c(cis,paste('(',ciu,', ',cil,')',sep=''))

#### exchangeable working correlation matrix

In [25]:
ipw_ex <- geem(formula = y~LIT_grp,
               id = school_id, data = dat1,
               family =  binomial("logit"),
               weights = dat1$weight,
               corstr = "exchangeable")
summary(ipw_ex)

            Estimates Model SE Robust SE    wald        p
(Intercept)  -0.07491   0.1290    0.1182 -0.6336 0.526300
LIT_grpyes    0.58800   0.1879    0.1923  3.0570 0.002234

 Estimated Correlation Parameter:  0.176 
 Correlation Structure:  exchangeable 
 Est. Scale Parameter:  8.773 

 Number of GEE iterations: 3 
 Number of Clusters:  101    Maximum Cluster Size:  27 
 Number of observations with nonzero weight:  2170 

In [26]:
# save the results 
m <- summary(ipw_ex)$beta[2]
s <- summary(ipw_ex)$se.robust[2]
p <- summary(ipw_ex)$p[2]
ciu <- m-1.96*s; cil <- m+1.96*s

# save results 
means <- c(means,exp(m))
sds <- c(sds,s)
ps <- c(ps,p)
ciu <- formatC(exp(ciu),2,format="f"); cil <- formatC(exp(cil),2,format="f")
cis <- c(cis,paste('(',ciu,', ',cil,')',sep=''))

## CW-GEE

#### independent working correlation matrix

In [27]:
ipw_clu_ind <- geem(formula = y~LIT_grp,
                 id=school_id, data = dat1,
                 family =  binomial("logit"),
                 weights = dat1$weight2,
                 corstr = "independence")
summary(ipw_clu_ind)

            Estimates Model SE Robust SE    wald        p
(Intercept)  -0.06748  0.05981    0.1228 -0.5495 0.582600
LIT_grpyes    0.60200  0.08759    0.1907  3.1570 0.001592

 Estimated Correlation Parameter:  0 
 Correlation Structure:  independence 
 Est. Scale Parameter:  9.537 

 Number of GEE iterations: 2 
 Number of Clusters:  101    Maximum Cluster Size:  27 
 Number of observations with nonzero weight:  2170 

In [28]:
# save the results 
m <- summary(ipw_clu_ind)$beta[2]
s <- summary(ipw_clu_ind)$se.robust[2]
p <- summary(ipw_clu_ind)$p[2]
ciu <- m-1.96*s; cil <- m+1.96*s

# save results 
means <- c(means,exp(m))
sds <- c(sds,s)
ps <- c(ps,p)
ciu <- formatC(exp(ciu),2,format="f"); cil <- formatC(exp(cil),2,format="f")
cis <- c(cis,paste('(',ciu,', ',cil,')',sep=''))

#### exchangeable working correlation matrix

In [29]:
ipw_clu_ex <- geem(formula = y~LIT_grp,
                id = school_id, data = dat1,
                family =  binomial("logit"),
                weights = dat1$weight2,
                corstr = "exchangeable")
summary(ipw_clu_ex)

            Estimates Model SE Robust SE    wald        p
(Intercept)  -0.07377   0.1287    0.1216 -0.6067 0.544000
LIT_grpyes    0.60320   0.1875    0.1890  3.1920 0.001414

 Estimated Correlation Parameter:  0.1714 
 Correlation Structure:  exchangeable 
 Est. Scale Parameter:  9.533 

 Number of GEE iterations: 3 
 Number of Clusters:  101    Maximum Cluster Size:  27 
 Number of observations with nonzero weight:  2170 

In [30]:
# save the results 
m <- summary(ipw_clu_ex)$beta[2]
s <- summary(ipw_clu_ex)$se.robust[2]
p <- summary(ipw_clu_ex)$p[2]
ciu <- m-1.96*s; cil <- m+1.96*s

# save results 
means <- c(means,exp(m))
sds <- c(sds,s)
ps <- c(ps,p)
ciu <- formatC(exp(ciu),2,format="f"); cil <- formatC(exp(cil),2,format="f")
cis <- c(cis,paste('(',ciu,', ',cil,')',sep=''))

## MMI-GEE

In [31]:
covariates <- c("BL_gll21_total","age_child","sex","schlevel_comp","BL_ses")
data.miss <- dat1
# data frame for response variables with missing values
y.cat <- data.frame(outcome=factor(data.miss$y))  
# number of levels in outcome variable
y.numcat <- c(2)     
# data frame for clusters
clus <- data.frame(clus=data.miss$school_id)         
# covariates, includes 1s.
nobs <- dim(data.miss)[1]
x <- data.frame(rep(1,nobs),
                data.miss[,covariates],
                group=data.miss$LIT_grp)

#### generate the complete dataset

* run to generate Nimp full datasets

* imp is full datasets 


In [32]:
set.seed(123)
Nimp <- 15
imp <- jomo1rancat(Y.cat = y.cat, Y.numcat = y.numcat, X = x,
                  clus = clus, nburn = 100, nbetween = 25, 
                  nimp = Nimp,output = 0)

#### save the mean and variance results

In [33]:
mmi_est_ind <- c();mmi_est_ex <- c()
mmi_std_ind <- c();mmi_std_ex <- c()
mmi_p_ind <- c();mmi_p_ex <- c()

for(i in 1:Nimp){
  temp <- imp[imp$Imputation==i,]
  rownames(temp) <- NULL
  temp$outcome <- as.numeric(temp$outcome)-1
  
  # run the analysis for Nimp times
  mmi_ind <- geem(formula=outcome~group,
               id=clus , data = temp,
               family =  binomial("logit"),
               corstr = "independence")
  mmi_ex <- geem(formula=outcome~group,
              id=clus , data = temp,
              family =  binomial("logit"),
              corstr = "exchangeable")
  
  # save the results 
  mmi_est_ind <- c(mmi_est_ind,
                summary(mmi_ind)$beta[2])
  mmi_std_ind <- c(mmi_std_ind,
                summary(mmi_ind)$se.robust[2])
  
  # save the results 
  mmi_est_ex <- c(mmi_est_ex,
               summary(mmi_ex)$beta[2])
  mmi_std_ex <- c(mmi_std_ex,
               summary(mmi_ex)$se.robust[2])
  mmi_p_ind <- c(mmi_p_ind,summary(mmi_ind)$p[2])
  mmi_p_ex <- c(mmi_p_ex,summary(mmi_ex)$p[2])
}

#### pool the results

In [34]:
mypool <- function(mean0,sd0,num=5,J=50){
    # input: 
    # mean0: a vector of the values of estimated beta parameter, with length equals to the imputation time.
    # sd0: a vector of the standard deviations of the estimated beta parameter, with length equals to the imputation time.
    # num: impuation time
    # J: the number of clusters in each intervention arm 
  
  # count the times of NA in the input vector.
  na_times <- sum(is.na(mean0)) 
  # the number of imputations without NA. 
  num_actual <- num-na_times 
  
  # the MI estimate of the beta parameter
  m <- mean(mean0,na.rm=TRUE) 
  
  # estimate of average wihtin-imputation variance 
  # i.e. based on the SE^2 of the beta parameter from each fitted model
  W <- mean(sd0^2,na.rm=TRUE) 
  
  # estimate of between-imputation variance 
  # i.e. empirical SD of the point estimates of beta parameter
  B <- var(mean0,na.rm=TRUE) 

  # estimate of total variance 
  # i.e. will need to take the sqrt of this to use for inference in making confidence intervals etc.
  v_hat <- W+(1+1/num_actual)*B 
  v_hat_sqrt<-sqrt(v_hat)
  
  # Confindence interval
  # Testing based on naive normal distribution assumption
  l <- m-1.96*v_hat_sqrt
  u <- m+1.96*v_hat_sqrt
  
  # Testing based on standard results from MI literature
  # i.e. df of t distribution for testing based on standard results from MI literature
  df_denom <- (1+1/num_actual)*B
  df_part <- 1+W/df_denom
  df_t <- (num_actual-1)*df_part^2 # adjusted df of t distribution
  l_t <- m-qt(0.975,df_t)*v_hat_sqrt
  u_t <- m+qt(0.975,df_t)*v_hat_sqrt
  
  # Testing based on results from MMI literature, Barnard and Rubin (1999), 
  # df of t distribution for testing based on results in re adjustment for MMI feature
  
  #df for full data
  df_com <- 2*J - 2 
  
  # calculate the adjusted df based on the literature.
  parenthesis <- 1+df_denom*(1/W) 
  df_obs <- df_com*((df_com+1)/(df_com+3))*(1/parenthesis) 
  df_adj_t <- 1/(1/df_t + 1/df_obs) 
  
  # Print the results
  print("Standard t df and Barnard/Rubin adjusted t df");
  print(c(df_t, df_adj_t))
  print("97.5% quantiles from standard t df and Barnard/Rubin adjusted t df");
  print(c(qt(0.975,df_t), qt(0.975,df_adj_t)))
  
  # The confidence interval calculated based on df of standard t distribution and adjusted df of t distribution 
  l_t <- m-qt(0.975,df_t)*v_hat_sqrt
  u_t <- m+qt(0.975,df_t)*v_hat_sqrt
  l_adj_t <- m-qt(0.975,df_adj_t)*v_hat_sqrt
  u_adj_t <- m+qt(0.975,df_adj_t)*v_hat_sqrt
  
  return(list(mean=m,std=v_hat_sqrt,
              df_t=df_t,
              df_adj_t=df_adj_t))
}

In [35]:
pool1 <- mypool(mmi_est_ind,mmi_std_ind,num=Nimp)
pool2 <- mypool(mmi_est_ex,mmi_std_ex,num=Nimp)

[1] "Standard t df and Barnard/Rubin adjusted t df"
[1] 73829.75487    94.61522
[1] "97.5% quantiles from standard t df and Barnard/Rubin adjusted t df"
[1] 1.959996 1.985355
[1] "Standard t df and Barnard/Rubin adjusted t df"
[1] 75391.54784    94.63147
[1] "97.5% quantiles from standard t df and Barnard/Rubin adjusted t df"
[1] 1.959995 1.985351


In [36]:
# save the results 
# independent working correlation matrix
ciu <- pool1$mean - qt(0.975, pool1$df_adj_t)*pool1$std;
cil <- pool1$mean + qt(0.975, pool1$df_adj_t)*pool1$std
means <- c(means,exp(pool1$mean))
ps <- c(ps,round(mean(mmi_p_ind),3))
sds <- c(sds,pool1$std)
ciu <- formatC(exp(ciu),2,format="f"); cil <- formatC(exp(cil),2,format="f")
cis <- c(cis,paste('(',ciu,', ',cil,')',sep=''))

# exchangeable working correlation matrix
ciu <- pool2$mean - qt(0.975, pool2$df_adj_t)*pool2$std;
cil <- pool2$mean + qt(0.975, pool2$df_adj_t)*pool2$std
# save the results
means <- c(means,exp(pool2$mean))
sds <- c(sds,pool2$std) 
ps <- c(ps,round(mean(mmi_p_ex),3))
ciu <- formatC(exp(ciu),2,format="f"); cil <- formatC(exp(cil),2,format="f")
cis <- c(cis,paste('(',ciu,', ',cil,')',sep=''))

## Draw table 1b



In [37]:
# use the saved resutls to draw summary table

coefs_ind <- formatC(means[seq(1,10,2)],2,format="f")
coefs_ex <- formatC(means[seq(2,10,2)],2,format="f")
std_ind <- formatC(sds[seq(1,10,2)],2,format="f")
std_ex <- formatC(sds[seq(2,10,2)],2,format="f")
cis_ind <- formatC(cis[seq(1,10,2)],2,format="f")
cis_ex <- formatC(cis[seq(2,10,2)],2,format="f")
p_ind <- formatC(ps[seq(1,10,2)],3,format="f")
p_ex <- formatC(ps[seq(2,10,2)],3,format="f")
names <- c('CRA-GEE','A-CRA-GEE','W-GEE','CW-GEE','MMI-GEE')

In [45]:
table_sum <- cbind(names,coefs_ex,std_ex,cis_ex,p_ex,coefs_ind,std_ind,cis_ind,p_ind)
colnames(table_sum) <- c('Method','Estimated','SE','Confidence Interval','p-value',
                        'Estimated','SE','Confidence Interval','p-value')

In [46]:
table_sum[table_sum[,c(5)] == "0.000",5] <- '<0.001'
table_sum[table_sum[,c(9)] == "0.000",9] <- '<0.001'
table_sum

Method,Estimated,SE,Confidence Interval,p-value,Estimated,SE,Confidence Interval,p-value
CRA-GEE,1.82,0.19,"(1.26, 2.65)",0.002,1.84,0.19,"(1.27, 2.66)",0.001
A-CRA-GEE,1.93,0.17,"(1.38, 2.71)",<0.001,1.92,0.17,"(1.37, 2.68)",<0.001
W-GEE,1.80,0.19,"(1.23, 2.62)",0.002,1.81,0.19,"(1.24, 2.63)",0.002
CW-GEE,1.83,0.19,"(1.26, 2.65)",0.001,1.83,0.19,"(1.26, 2.65)",0.002
MMI-GEE,1.81,0.19,"(1.25, 2.63)",0.002,1.79,0.19,"(1.23, 2.60)",0.002


In [52]:
hali_ex <- table_sum[,1:5]
hali_ex

Method,Estimated,SE,Confidence Interval,p-value
CRA-GEE,1.82,0.19,"(1.26, 2.65)",0.002
A-CRA-GEE,1.93,0.17,"(1.38, 2.71)",<0.001
W-GEE,1.80,0.19,"(1.23, 2.62)",0.002
CW-GEE,1.83,0.19,"(1.26, 2.65)",0.001
MMI-GEE,1.81,0.19,"(1.25, 2.63)",0.002


In [53]:
table_s1 <- table_sum
table_1b <- hali_ex